# Dark Souls II Reviews (2025)

In [1]:
from get_data import *

In [2]:
reviews = get_data()

In [3]:
reviews.update_date = pd.to_datetime(reviews["update_date"], unit='s')#.dt.date

## Looking at reviews after DLC release:

In [4]:
dlc = reviews[reviews.review.str.contains(r'\bdlc\b')]

In [5]:
dlc.voted_up.value_counts(normalize=True)

True     0.808089
False    0.191911
Name: voted_up, dtype: float64

- Crown of the Sunken King DLC launched: 7/22/2014
- Crown of the Old Iron King DLC launched: 8/26/2014
- Crown of the Ivory King DLC launched: 9/30/2014
- Scholar of the First Sin Edition launched: 4/1/2015

In [6]:
dlc1_start = pd.Timestamp("2014-07-22")
dlc2_start = pd.Timestamp("2014-08-26")
dlc3_start = pd.Timestamp("2014-09-30")
sotfs_start = pd.Timestamp("2015-04-01")

dlc1 = reviews[(reviews["update_date"] >= dlc1_start) & (reviews["update_date"] < dlc2_start)]
dlc2 = reviews[(reviews["update_date"] >= dlc2_start) & (reviews["update_date"] < dlc3_start)]
dlc3 = reviews[(reviews["update_date"] >= dlc3_start) & (reviews["update_date"] < sotfs_start)]

In [7]:
display(dlc1, dlc2, dlc3)

,review,language,init_date,update_date,in_early_access,voted_up,month_name,month,year,day
recommendationid,,,,,,,,,,


,review,language,init_date,update_date,in_early_access,voted_up,month_name,month,year,day
recommendationid,,,,,,,,,,


,review,language,init_date,update_date,in_early_access,voted_up,month_name,month,year,day
recommendationid,,,,,,,,,,
